# Comparison Notebook

The last few cells of code including the big pandas dataframe compare different players

In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

from game import Game
from player import Player

In [2]:
from your_name import YourName

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(11, 1), (9, 0), (12, 1), (10, 1), (14, 3), (12, 2), (10, 3), (13, 2)]
[(9, 3), (13, 1), (14, 1), (14, 0), (11, 2), (14, 2), (9, 1), (10, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 10000 #For most accurate tests
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                #print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 YourName
==== 1 YourName


In [9]:
full_results

[[None,
  [[10000, 0],
   [0, 0],
   [8.0, 7.5025],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.5025],
   [2.6412, 0.0],
   [0, 0],
   0,
   0]],
 [None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM YourName 
0
RANDOM - 10000 
1
YourName 0 - 
Draws
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 
Moves
 RANDOM YourName 
0
RANDOM - 8.0 
1
YourName 7.5025 - 
Cards
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 15.0 - 
Checks
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 0.0 - 
Draw Decisions
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 7.5025 - 
Cheats
 RANDOM YourName 
0
RANDOM - 2.6412 
1
YourName 0.0 - 
Errors
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 


### __________________________________________My Code ______________________________________________

In [11]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["RandomPlayer2", "RANDOM", RandomPlayer]]

def compare_all(agent_strategies, verbose = 1):
    wins = {agent_strategies[i][0]: {agent_strategies[i][0]: None} for i in range(len(agent_strategies))}
    for i in range(len(agent_strategies)):
        for ii in range(i+1, len(agent_strategies)):
            player1 = agent_strategies[i]
            player2 = agent_strategies[ii]
            strategy = [player1, player2]
            for A in range(len(strategy)):
                for B in range(A+1,len(strategy)):
                    results = comparePlayers(strategy[A][2], strategy[B][2])
                    full_results[A][B] = results
                    wins[strategy[A][0]][strategy[B][0]] = full_results[0][1][0][1]
                    wins[strategy[B][0]][strategy[A][0]] = full_results[0][1][0][0]
                    if verbose:
                        if full_results[0][1][8][0] > 0 or full_results[0][1][8][1] > 0 or full_results[0][1][9] > 0:
                            print(f"Errors occured in game: {strategy[A][0]} vs {strategy[B][0]}")
    df = pd.DataFrame(wins)
    df['|| Total Wins ||'] = df.fillna(0).sum(axis=1)
    df = df.sort_values(by='|| Total Wins ||', ascending=False)

    dominated_players = []
    temp_df = df.drop(columns=["|| Total Wins ||"])
    for i, row in temp_df.iterrows():
        num_dominated = 0
        for column in temp_df.columns:
            if i != column and temp_df[column][i] > temp_df[i][column]:
                num_dominated += 1
        dominated_players.append(num_dominated)
    df['|| Dominated Players ||'] = dominated_players
    return df

compare_all(agent_strategies)

,RandomPlayer,YourName,RandomPlayer2,|| Total Wins ||,|| Dominated Players ||
RandomPlayer2,5002.0,10000.0,NaN,15002.0,2
RandomPlayer,NaN,10000.0,4998.0,14998.0,1
YourName,0.0,NaN,0.0,0.0,0


## Player Comaprison

Below I have tested the players against each other, and different simple heuristics as well as my solutions. The dataframe orders players according to the total number of wins against remaining players ( the "|| Total Wins ||" column), top players are the best. However, you can also check how many players the agent was better than - "|| Dominated Players ||". The numbers in remaining columns show how many times your agent has won against someone, so if you look at the row from dataframe above as an example the agent called "RandomPlayer" and compare it with column "RandomPlayer2" you can see that RandomPlayer won against the RandomPlayer2 4968 games out of 10000, you can also see that RandomPlayer2 won 5032 games against RandomPlayer, etc.

Below the dataframe I also print for some most important players their row values to see which agents they won against and which they won against, etc.

In [12]:
from Test_Players import *
from B import *
from adamK import *
from Adam_player import *
from ensamble import *
from Michal_Janek_ver_1 import *
from Michal_Janek_ver_2 import *
from Vadym_Player import *
from Krzysztof_Player import *
from Samuel_Player import *
from pukacki_tabaszewski import *

In [13]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Saint_Sceptic_25_Naive", "SAINT", Saint_Sceptic_25_Naive],
            ["Saint_Sceptic_50_Naive", "SAINT", Saint_Sceptic_50_Naive],
            ["Saint_Sceptic_75_Naive", "SAINT", Saint_Sceptic_75_Naive],
            ["Saint_Sceptic_Adaptable_Naive", "SAINT", Saint_Sceptic_Adaptable_Naive],
            ["Mat_Player_ver_4", "ENSAMBLE", ensamble_player],
            ["YourName - not implemented Player", "YourName", YourName],
           ["MyRandom_Default", "CUSTOM_RANDOM", MyRandom_Default],
           ["MyRandom_All_Cards", "CUSTOM_RANDOM", MyRandom_All_Cards],
           ["MyRandom_Real_Cards", "CUSTOM_RANDOM", MyRandom_Real_Cards],
           ["Random_Never_Accuse", "CUSTOM_RANDOM", Random_Never_Accuse],
           ["Random_Always_Accuse", "CUSTOM_RANDOM", Random_Always_Accuse],
           ["Mimic", "CUSTOM_RANDOM", Mimic],
           ["Saint_Believer", "SAINT", Saint_Believer],
           ["Saint_Once_in_a_While", "SAINT", Saint_Once_in_a_While],
           ["Saint_Two_Face", "SAINT", Saint_Two_Face],
           ["Saint_Accountant", "SAINT", Saint_Accountant],
           ["Saint_Nervous", "SAINT", Saint_Nervous],
           ["Mat_Player_ver_1", "SAINT", Saint_Nervous_ver_2],
           ["Mat_Player_ver_2", "SAINT", Saint_Nervous_ver_3],
           ["Mat_Player_ver_3", "SAINT", Saint_Nervous_Memory],
           ["Adam_M_Player", "CUSTOM", Adam_player],
           ["AdamK_ver_1", "CUSTOM", AdamK],
           ["AdamK2_ver_2", "CUSTOM", AdamK2],
           ["Saint_Collector", "SAINT", Saint_Collector],
           ["Dabbler_Believer", "DABBLER", Dabbler_Believer],
           ["Dabbler_Once_in_a_While", "DABBLER", Dabbler_Once_in_a_While],
           ["Dabbler_Two_Face", "DABBLER", Dabbler_Two_Face],
           ["Dabbler_Accountant", "DABBLER", Dabbler_Accountant],
           ["Dabbler_Nervous", "DABBLER", Dabbler_Nervous],
           ["Dabbler_Collector", "DABBLER", Dabbler_Collector],
           ["Coin_Flipper_Believer", "COIN_FLIPPER", Coin_Flipper_Believer],
           ["Coin_Flipper_Once_in_a_While", "COIN_FLIPPER", Coin_Flipper_Once_in_a_While],
           ["Coin_Flipper_Two_Face", "COIN_FLIPPER", Coin_Flipper_Two_Face],
           ["Coin_Flipper_Accountant", "COIN_FLIPPER", Coin_Flipper_Accountant],
           ["Coin_Flipper_Nervous", "COIN_FLIPPER", Coin_Flipper_Nervous],
           ["Coin_Flipper_Collector", "COIN_FLIPPER", Coin_Flipper_Collector],
           ["Pathological_Believer", "PATHOLOGICAL", Pathological_Believer],
           ["Pathological_Once_in_a_While", "PATHOLOGICAL", Pathological_Once_in_a_While],
           ["Pathological_Two_Face", "PATHOLOGICAL", Pathological_Two_Face],
           ["Pathological_Accountant", "PATHOLOGICAL", Pathological_Accountant],
           ["Pathological_Nervous", "PUSHER", Pathological_Nervous],
           ["Pathological_Collector", "PUSHER", Pathological_Collector],
           ["Pusher_Believer", "PUSHER", Pusher_Believer],
           ["Pusher_Once_in_a_While", "PUSHER", Pusher_Once_in_a_While],
           ["Pusher_Two_Face", "PUSHER", Pusher_Two_Face],
           ["Pusher_Accountant", "PUSHER", Pusher_Accountant],
           ["Pusher_Nervous", "PUSHER", Pusher_Nervous],
           ["Pusher_Collector", "PUSHER", Pusher_Collector],
           ["B_1", "B_PLAYER", Blayer],
           ["B_2", "B_PLAYER", Blayer2],
           ["B_3", "B_PLAYER", Blayer3],
           ["Bartek_ver_1", "B_PLAYER", Saint_Nervous_ver_B],
           ["Bartek_ver_2", "B_PLAYER", Saint_Nervous_ver_B2],
           ["Bartek_ver_3", "B_PLAYER", Saint_Nervous_ver_B3],
           ["Bartek_ver_4", "B_PLAYER", Saint_Nervous_ver_B4],
           ["Janek_Michal_ver_1", "MICHAL_JANEK", KaminskiIndrzejczak_ver_1],
           ["Janek_Michal_ver_2", "MICHAL_JANEK", KaminskiIndrzejczak_ver_2],
           ["Vadym_Player", "VADYM", Vadym_Player],
           ["Krzysztof_Player", "KRZYSZTOF", Krzysztof_Player],
           ["Samuel_Player", "SAMUEL", Amalgamat],
           ["Pukacki_Tabaszewski", "PUKACKI_TABASZEWSKI", Pukacki_Tabaszewski]
           ]

results = compare_all(agent_strategies)
results

,RandomPlayer,Saint_Sceptic_25_Naive,Saint_Sceptic_50_Naive,Saint_Sceptic_75_Naive,Saint_Sceptic_Adaptable_Naive,Mat_Player_ver_4,YourName - not implemented Player,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,...,Bartek_ver_3,Bartek_ver_4,Janek_Michal_ver_1,Janek_Michal_ver_2,Vadym_Player,Krzysztof_Player,Samuel_Player,Pukacki_Tabaszewski,|| Total Wins ||,|| Dominated Players ||
Bartek_ver_1,9999.0,10000.0,10000.0,10000.0,8279.0,4948.0,10000.0,10000.0,10000.0,9997.0,...,5408.0,5344.0,10000.0,3939.0,9212.0,10000.0,4053.0,5054.0,516121.0,55
Bartek_ver_2,9999.0,10000.0,10000.0,10000.0,8289.0,5015.0,10000.0,10000.0,10000.0,9997.0,...,5396.0,5412.0,10000.0,3941.0,9217.0,10000.0,4082.0,4958.0,515945.0,57
Mat_Player_ver_4,9998.0,10000.0,10000.0,10000.0,8310.0,NaN,10000.0,10000.0,10000.0,9999.0,...,5289.0,5411.0,10000.0,4005.0,9236.0,10000.0,4063.0,4967.0,515714.0,55
Pukacki_Tabaszewski,10000.0,10000.0,10000.0,10000.0,8325.0,5033.0,10000.0,10000.0,10000.0,9998.0,...,5321.0,5419.0,10000.0,3943.0,9229.0,10000.0,3984.0,NaN,515699.0,55
Mat_Player_ver_2,9998.0,10000.0,10000.0,10000.0,8164.0,5011.0,10000.0,9999.0,10000.0,9999.0,...,4503.0,4436.0,10000.0,4070.0,9087.0,10000.0,4064.0,5007.0,510507.0,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pathological_Collector,227.0,8.0,88.0,1611.0,740.0,75.0,10000.0,327.0,356.0,261.0,...,340.0,401.0,116.0,1353.0,675.0,0.0,1291.0,91.0,106951.0,8
Random_Never_Accuse,220.0,13.0,135.0,2065.0,664.0,80.0,10000.0,228.0,242.0,241.0,...,291.0,281.0,103.0,1293.0,536.0,0.0,1458.0,73.0,105967.0,9
Pathological_Believer,233.0,8.0,163.0,2118.0,647.0,89.0,10000.0,246.0,206.0,226.0,...,263.0,274.0,134.0,1197.0,565.0,0.0,1431.0,104.0,105938.0,10
Mimic,119.0,0.0,0.0,0.0,1817.0,0.0,10000.0,69.0,168.0,82.0,...,70.0,77.0,0.0,57.0,14.0,0.0,0.0,0.0,66023.0,2


In [14]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
results

,RandomPlayer,Saint_Sceptic_25_Naive,Saint_Sceptic_50_Naive,Saint_Sceptic_75_Naive,Saint_Sceptic_Adaptable_Naive,Mat_Player_ver_4,YourName - not implemented Player,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,Random_Never_Accuse,Random_Always_Accuse,Mimic,Saint_Believer,Saint_Once_in_a_While,Saint_Two_Face,Saint_Accountant,Saint_Nervous,Mat_Player_ver_1,Mat_Player_ver_2,Mat_Player_ver_3,Adam_M_Player,AdamK_ver_1,AdamK2_ver_2,Saint_Collector,Dabbler_Believer,Dabbler_Once_in_a_While,Dabbler_Two_Face,Dabbler_Accountant,Dabbler_Nervous,Dabbler_Collector,Coin_Flipper_Believer,Coin_Flipper_Once_in_a_While,Coin_Flipper_Two_Face,Coin_Flipper_Accountant,Coin_Flipper_Nervous,Coin_Flipper_Collector,Pathological_Believer,Pathological_Once_in_a_While,Pathological_Two_Face,Pathological_Accountant,Pathological_Nervous,Pathological_Collector,Pusher_Believer,Pusher_Once_in_a_While,Pusher_Two_Face,Pusher_Accountant,Pusher_Nervous,Pusher_Collector,B_1,B_2,B_3,Bartek_ver_1,Bartek_ver_2,Bartek_ver_3,Bartek_ver_4,Janek_Michal_ver_1,Janek_Michal_ver_2,Vadym_Player,Krzysztof_Player,Samuel_Player,Pukacki_Tabaszewski,|| Total Wins ||,|| Dominated Players ||
Bartek_ver_1,9999.0,10000.0,10000.0,10000.0,8279.0,4948.0,10000.0,10000.0,10000.0,9997.0,9923.0,10000.0,10000.0,6095.0,7794.0,9967.0,9426.0,4968.0,4969.0,5079.0,7551.0,9987.0,6514.0,6788.0,6101.0,8729.0,9472.0,9996.0,9881.0,8410.0,8726.0,9506.0,9808.0,9999.0,9980.0,9479.0,9561.0,9901.0,9975.0,10000.0,9994.0,9946.0,9923.0,9380.0,9716.0,9998.0,9955.0,9395.0,9597.0,6521.0,6539.0,5413.0,NaN,4926.0,5408.0,5344.0,10000.0,3939.0,9212.0,10000.0,4053.0,5054.0,516121.0,55
Bartek_ver_2,9999.0,10000.0,10000.0,10000.0,8289.0,5015.0,10000.0,10000.0,10000.0,9997.0,9910.0,10000.0,10000.0,5966.0,7828.0,9971.0,9430.0,4961.0,5011.0,5034.0,7532.0,9987.0,6387.0,6778.0,6050.0,8672.0,9497.0,9998.0,9873.0,8430.0,8736.0,9470.0,9807.0,9999.0,9976.0,9491.0,9547.0,9915.0,9971.0,10000.0,9997.0,9924.0,9922.0,9378.0,9721.0,9997.0,9952.0,9336.0,9550.0,6589.0,6503.0,5469.0,5074.0,NaN,5396.0,5412.0,10000.0,3941.0,9217.0,10000.0,4082.0,4958.0,515945.0,57
Mat_Player_ver_4,9998.0,10000.0,10000.0,10000.0,8310.0,NaN,10000.0,10000.0,10000.0,9999.0,9920.0,10000.0,10000.0,6021.0,7780.0,9969.0,9434.0,5052.0,4976.0,4989.0,7455.0,9988.0,6458.0,6821.0,6025.0,8628.0,9483.0,10000.0,9875.0,8405.0,8708.0,9517.0,9810.0,10000.0,9976.0,9482.0,9569.0,9911.0,9956.0,9998.0,9998.0,9926.0,9925.0,9385.0,9720.0,10000.0,9941.0,9354.0,9535.0,6400.0,6544.0,5465.0,5052.0,4985.0,5289.0,5411.0,10000.0,4005.0,9236.0,10000.0,4063.0,4967.0,515714.0,55
Pukacki_Tabaszewski,10000.0,10000.0,10000.0,10000.0,8325.0,5033.0,10000.0,10000.0,10000.0,9998.0,9927.0,10000.0,10000.0,5970.0,7729.0,9957.0,9458.0,4943.0,4920.0,4993.0,7462.0,9988.0,6550.0,6770.0,6017.0,8626.0,9489.0,9997.0,9886.0,8465.0,8741.0,9511.0,9837.0,9999.0,9976.0,9470.0,9581.0,9896.0,9961.0,10000.0,9997.0,9914.0,9909.0,9390.0,9747.0,10000.0,9952.0,9373.0,9581.0,6572.0,6461.0,5444.0,4946.0,5042.0,5321.0,5419.0,10000.0,3943.0,9229.0,10000.0,3984.0,NaN,515699.0,55
Mat_Player_ver_2,9998.0,10000.0,10000.0,10000.0,8164.0,5011.0,10000.0,9999.0,10000.0,9999.0,9906.0,10000.0,9614.0,5898.0,7668.0,9969.0,9403.0,5071.0,5005.0,NaN,7426.0,9993.0,6495.0,6311.0,5967.0,8430.0,9377.0,9997.0,9831.0,8184.0,8554.0,9422.0,9779.0,9999.0,9967.0,9410.0,9443.0,9871.0,9946.0,10000.0,9993.0,9893.0,9920.0,9370.0,9722.0,9997.0,9943.0,9376.0,9512.0,6526.0,6515.0,4579.0,4921.0,4966.0,4503.0,4436.0,10000.0,4070.0,9087.0,10000.0,4064.0,5007.0,510507.0,54
Mat_Player_ver_1,9999.0,10000.0,10000.0,10000.0,8211.0,5024.0,10000.0,9999.0,10000.0,9999.0,9901.0,10000.0,9630.0,5874.0,7621.0,9966.0,9428.0,4905.0,NaN,4995.0,7523.0,9988.0,6354.0,6297.0,5888.0,8465.0,9329.0,9995.0,9829.0,8194.0,8533.0,9408.0,9768.0,9998.0,9969.0,9387.0,9461.0,9897.0,9964.0,10000.0,9997.0,9901.0,9892.0,9330.0,9696.0,9996.0,9939.0,9358.0,9506.0,6601.0,6532.0,4604.0,5031.0,4989.0,4610.0,4431.0,10000.0,3982.0,9124.0,10000.0,4016.0,5080.0,510414.0,53
Saint_Nervous,9999

### Player-Wise Comparisons

In [15]:
for col, val in results.loc["Janek_Michal_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9831.0
Saint_Sceptic_25_Naive: 5645.0
Saint_Sceptic_50_Naive: 5591.0
Saint_Sceptic_75_Naive: 5632.0
Saint_Sceptic_Adaptable_Naive: 0.0
Mat_Player_ver_4: 0.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9991.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9692.0
Random_Never_Accuse: 9897.0
Random_Always_Accuse: 9997.0
Mimic: 10000.0
Saint_Believer: 511.0
Saint_Once_in_a_While: 18.0
Saint_Two_Face: 0.0
Saint_Accountant: 0.0
Saint_Nervous: 0.0
Mat_Player_ver_1: 0.0
Mat_Player_ver_2: 0.0
Mat_Player_ver_3: 2514.0
Adam_M_Player: 5735.0
AdamK_ver_1: 290.0
AdamK2_ver_2: 2230.0
Saint_Collector: 578.0
Dabbler_Believer: 3040.0
Dabbler_Once_in_a_While: 3636.0
Dabbler_Two_Face: 6268.0
Dabbler_Accountant: 5315.0
Dabbler_Nervous: 2035.0
Dabbler_Collector: 3244.0
Coin_Flipper_Believer: 8552.0
Coin_Flipper_Once_in_a_While: 8911.0
Coin_Flipper_Two_Face: 9826.0
Coin_Flipper_Accountant: 9372.0
Coin_Flipper_Nervous: 8061.0
Coin_Flipper_Collector: 8548.0
Pathological_Believe

In [16]:
for col, val in results.loc["Janek_Michal_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 6462.0
Saint_Sceptic_25_Naive: 9108.0
Saint_Sceptic_50_Naive: 9245.0
Saint_Sceptic_75_Naive: 9317.0
Saint_Sceptic_Adaptable_Naive: 8487.0
Mat_Player_ver_4: 5995.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8098.0
MyRandom_All_Cards: 9631.0
MyRandom_Real_Cards: 4247.0
Random_Never_Accuse: 8707.0
Random_Always_Accuse: 5651.0
Mimic: 9943.0
Saint_Believer: 7570.0
Saint_Once_in_a_While: 7478.0
Saint_Two_Face: 8043.0
Saint_Accountant: 7836.0
Saint_Nervous: 6028.0
Mat_Player_ver_1: 6018.0
Mat_Player_ver_2: 5930.0
Mat_Player_ver_3: 6123.0
Adam_M_Player: 7995.0
AdamK_ver_1: 5934.0
AdamK2_ver_2: 6607.0
Saint_Collector: 7453.0
Dabbler_Believer: 8848.0
Dabbler_Once_in_a_While: 8575.0
Dabbler_Two_Face: 7018.0
Dabbler_Accountant: 7057.0
Dabbler_Nervous: 6175.0
Dabbler_Collector: 8890.0
Coin_Flipper_Believer: 8834.0
Coin_Flipper_Once_in_a_While: 8386.0
Coin_Flipper_Two_Face: 6515.0
Coin_Flipper_Accountant: 6262.0
Coin_Flipper_Nervous: 6181.0
Coin_Flipper_Collector: 876

In [17]:
for col, val in results.loc["Vadym_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 6202.0
Saint_Sceptic_25_Naive: 4267.0
Saint_Sceptic_50_Naive: 4927.0
Saint_Sceptic_75_Naive: 5774.0
Saint_Sceptic_Adaptable_Naive: 6964.0
Mat_Player_ver_4: 764.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8271.0
MyRandom_All_Cards: 9858.0
MyRandom_Real_Cards: 4691.0
Random_Never_Accuse: 9464.0
Random_Always_Accuse: 974.0
Mimic: 9986.0
Saint_Believer: 7288.0
Saint_Once_in_a_While: 6441.0
Saint_Two_Face: 4425.0
Saint_Accountant: 5081.0
Saint_Nervous: 848.0
Mat_Player_ver_1: 876.0
Mat_Player_ver_2: 913.0
Mat_Player_ver_3: 914.0
Adam_M_Player: 3245.0
AdamK_ver_1: 5785.0
AdamK2_ver_2: 6393.0
Saint_Collector: 6994.0
Dabbler_Believer: 9261.0
Dabbler_Once_in_a_While: 8982.0
Dabbler_Two_Face: 6170.0
Dabbler_Accountant: 4862.0
Dabbler_Nervous: 4707.0
Dabbler_Collector: 9252.0
Coin_Flipper_Believer: 9326.0
Coin_Flipper_Once_in_a_While: 9136.0
Coin_Flipper_Two_Face: 6233.0
Coin_Flipper_Accountant: 4588.0
Coin_Flipper_Nervous: 5332.0
Coin_Flipper_Collector: 9380.0
Pa

In [18]:
for col, val in results.loc["Krzysztof_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9914.0
Saint_Sceptic_25_Naive: 4981.0
Saint_Sceptic_50_Naive: 4987.0
Saint_Sceptic_75_Naive: 4992.0
Saint_Sceptic_Adaptable_Naive: 0.0
Mat_Player_ver_4: 0.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 10000.0
Random_Never_Accuse: 10000.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 0.0
Saint_Once_in_a_While: 0.0
Saint_Two_Face: 0.0
Saint_Accountant: 0.0
Saint_Nervous: 0.0
Mat_Player_ver_1: 0.0
Mat_Player_ver_2: 0.0
Mat_Player_ver_3: 822.0
Adam_M_Player: 5681.0
AdamK_ver_1: 300.0
AdamK2_ver_2: 301.0
Saint_Collector: 128.0
Dabbler_Believer: 2845.0
Dabbler_Once_in_a_While: 2955.0
Dabbler_Two_Face: 3851.0
Dabbler_Accountant: 4482.0
Dabbler_Nervous: 1535.0
Dabbler_Collector: 2662.0
Coin_Flipper_Believer: 8852.0
Coin_Flipper_Once_in_a_While: 9098.0
Coin_Flipper_Two_Face: 9916.0
Coin_Flipper_Accountant: 9616.0
Coin_Flipper_Nervous: 8640.0
Coin_Flipper_Collector: 8933.0
Pathological_Believer

In [19]:
for col, val in results.loc["Adam_M_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9981.0
Saint_Sceptic_25_Naive: 4264.0
Saint_Sceptic_50_Naive: 4318.0
Saint_Sceptic_75_Naive: 4323.0
Saint_Sceptic_Adaptable_Naive: 67.0
Mat_Player_ver_4: 12.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9973.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 86.0
Saint_Once_in_a_While: 56.0
Saint_Two_Face: 8.0
Saint_Accountant: 40.0
Saint_Nervous: 9.0
Mat_Player_ver_1: 12.0
Mat_Player_ver_2: 7.0
Mat_Player_ver_3: 841.0
Adam_M_Player: nan
AdamK_ver_1: 684.0
AdamK2_ver_2: 578.0
Saint_Collector: 75.0
Dabbler_Believer: 5324.0
Dabbler_Once_in_a_While: 6504.0
Dabbler_Two_Face: 9394.0
Dabbler_Accountant: 7717.0
Dabbler_Nervous: 3996.0
Dabbler_Collector: 5461.0
Coin_Flipper_Believer: 9147.0
Coin_Flipper_Once_in_a_While: 9551.0
Coin_Flipper_Two_Face: 9985.0
Coin_Flipper_Accountant: 9856.0
Coin_Flipper_Nervous: 9009.0
Coin_Flipper_Collector: 9228.0
Pathological_Believer

In [20]:
for col, va in results.loc["AdamK_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 32.0
Saint_Sceptic_25_Naive: 32.0
Saint_Sceptic_50_Naive: 32.0
Saint_Sceptic_75_Naive: 32.0
Saint_Sceptic_Adaptable_Naive: 32.0
Mat_Player_ver_4: 32.0
YourName - not implemented Player: 32.0
MyRandom_Default: 32.0
MyRandom_All_Cards: 32.0
MyRandom_Real_Cards: 32.0
Random_Never_Accuse: 32.0
Random_Always_Accuse: 32.0
Mimic: 32.0
Saint_Believer: 32.0
Saint_Once_in_a_While: 32.0
Saint_Two_Face: 32.0
Saint_Accountant: 32.0
Saint_Nervous: 32.0
Mat_Player_ver_1: 32.0
Mat_Player_ver_2: 32.0
Mat_Player_ver_3: 32.0
Adam_M_Player: 32.0
AdamK_ver_1: 32.0
AdamK2_ver_2: 32.0
Saint_Collector: 32.0
Dabbler_Believer: 32.0
Dabbler_Once_in_a_While: 32.0
Dabbler_Two_Face: 32.0
Dabbler_Accountant: 32.0
Dabbler_Nervous: 32.0
Dabbler_Collector: 32.0
Coin_Flipper_Believer: 32.0
Coin_Flipper_Once_in_a_While: 32.0
Coin_Flipper_Two_Face: 32.0
Coin_Flipper_Accountant: 32.0
Coin_Flipper_Nervous: 32.0
Coin_Flipper_Collector: 32.0
Pathological_Believer: 32.0
Pathological_Once_in_a_While: 32.0
Patholog

In [21]:
for col, val in results.loc["AdamK2_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 9252.0
Saint_Sceptic_25_Naive: 9559.0
Saint_Sceptic_50_Naive: 9537.0
Saint_Sceptic_75_Naive: 9625.0
Saint_Sceptic_Adaptable_Naive: 7874.0
Mat_Player_ver_4: 3179.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9568.0
MyRandom_All_Cards: 9957.0
MyRandom_Real_Cards: 8392.0
Random_Never_Accuse: 8670.0
Random_Always_Accuse: 9128.0
Mimic: 9989.0
Saint_Believer: 5963.0
Saint_Once_in_a_While: 6887.0
Saint_Two_Face: 8518.0
Saint_Accountant: 8444.0
Saint_Nervous: 3771.0
Mat_Player_ver_1: 3703.0
Mat_Player_ver_2: 3689.0
Mat_Player_ver_3: 5352.0
Adam_M_Player: 9422.0
AdamK_ver_1: 4578.0
AdamK2_ver_2: nan
Saint_Collector: 6117.0
Dabbler_Believer: 8179.0
Dabbler_Once_in_a_While: 8359.0
Dabbler_Two_Face: 9343.0
Dabbler_Accountant: 8979.0
Dabbler_Nervous: 6747.0
Dabbler_Collector: 8144.0
Coin_Flipper_Believer: 8451.0
Coin_Flipper_Once_in_a_While: 8532.0
Coin_Flipper_Two_Face: 9253.0
Coin_Flipper_Accountant: 9020.0
Coin_Flipper_Nervous: 7407.0
Coin_Flipper_Collector: 8385.0

In [22]:
for col, val in results.loc["B_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 8630.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 6449.0
Mat_Player_ver_4: 3600.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9485.0
MyRandom_All_Cards: 9969.0
MyRandom_Real_Cards: 6251.0
Random_Never_Accuse: 3647.0
Random_Always_Accuse: 9927.0
Mimic: 10000.0
Saint_Believer: 4141.0
Saint_Once_in_a_While: 5799.0
Saint_Two_Face: 8865.0
Saint_Accountant: 8159.0
Saint_Nervous: 3507.0
Mat_Player_ver_1: 3399.0
Mat_Player_ver_2: 3474.0
Mat_Player_ver_3: 6429.0
Adam_M_Player: 9960.0
AdamK_ver_1: 4050.0
AdamK2_ver_2: 5273.0
Saint_Collector: 4269.0
Dabbler_Believer: 4028.0
Dabbler_Once_in_a_While: 4962.0
Dabbler_Two_Face: 9038.0
Dabbler_Accountant: 5653.0
Dabbler_Nervous: 3208.0
Dabbler_Collector: 4132.0
Coin_Flipper_Believer: 3859.0
Coin_Flipper_Once_in_a_While: 4555.0
Coin_Flipper_Two_Face: 8548.0
Coin_Flipper_Accountant: 5171.0
Coin_Flipper_Nervous: 3128.0
Coin_Flipper_Collector:

In [23]:
for col, val in results.loc["B_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 8196.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 6472.0
Mat_Player_ver_4: 3456.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9353.0
MyRandom_All_Cards: 9972.0
MyRandom_Real_Cards: 5051.0
Random_Never_Accuse: 3391.0
Random_Always_Accuse: 9911.0
Mimic: 10000.0
Saint_Believer: 4127.0
Saint_Once_in_a_While: 5670.0
Saint_Two_Face: 8863.0
Saint_Accountant: 8108.0
Saint_Nervous: 3429.0
Mat_Player_ver_1: 3468.0
Mat_Player_ver_2: 3485.0
Mat_Player_ver_3: 6438.0
Adam_M_Player: 9949.0
AdamK_ver_1: 4064.0
AdamK2_ver_2: 5195.0
Saint_Collector: 4181.0
Dabbler_Believer: 4043.0
Dabbler_Once_in_a_While: 4827.0
Dabbler_Two_Face: 8952.0
Dabbler_Accountant: 5604.0
Dabbler_Nervous: 2982.0
Dabbler_Collector: 3983.0
Coin_Flipper_Believer: 3786.0
Coin_Flipper_Once_in_a_While: 4393.0
Coin_Flipper_Two_Face: 8259.0
Coin_Flipper_Accountant: 4939.0
Coin_Flipper_Nervous: 2904.0
Coin_Flipper_Collector:

In [24]:
for col, val in results.loc["B_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 8909.0
Saint_Sceptic_25_Naive: 9230.0
Saint_Sceptic_50_Naive: 9262.0
Saint_Sceptic_75_Naive: 9266.0
Saint_Sceptic_Adaptable_Naive: 9095.0
Mat_Player_ver_4: 4535.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8963.0
MyRandom_All_Cards: 9861.0
MyRandom_Real_Cards: 7335.0
Random_Never_Accuse: 8709.0
Random_Always_Accuse: 7748.0
Mimic: 9871.0
Saint_Believer: 7503.0
Saint_Once_in_a_While: 7987.0
Saint_Two_Face: 9241.0
Saint_Accountant: 9424.0
Saint_Nervous: 6280.0
Mat_Player_ver_1: 5396.0
Mat_Player_ver_2: 5421.0
Mat_Player_ver_3: 5238.0
Adam_M_Player: 9399.0
AdamK_ver_1: 5249.0
AdamK2_ver_2: 5621.0
Saint_Collector: 7515.0
Dabbler_Believer: 8877.0
Dabbler_Once_in_a_While: 8787.0
Dabbler_Two_Face: 9210.0
Dabbler_Accountant: 9472.0
Dabbler_Nervous: 7279.0
Dabbler_Collector: 8889.0
Coin_Flipper_Believer: 8775.0
Coin_Flipper_Once_in_a_While: 8554.0
Coin_Flipper_Two_Face: 8931.0
Coin_Flipper_Accountant: 9318.0
Coin_Flipper_Nervous: 7185.0
Coin_Flipper_Collector: 884

In [25]:
for col, val in results.loc["Bartek_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8279.0
Mat_Player_ver_4: 4948.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9997.0
Random_Never_Accuse: 9923.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 6095.0
Saint_Once_in_a_While: 7794.0
Saint_Two_Face: 9967.0
Saint_Accountant: 9426.0
Saint_Nervous: 4968.0
Mat_Player_ver_1: 4969.0
Mat_Player_ver_2: 5079.0
Mat_Player_ver_3: 7551.0
Adam_M_Player: 9987.0
AdamK_ver_1: 6514.0
AdamK2_ver_2: 6788.0
Saint_Collector: 6101.0
Dabbler_Believer: 8729.0
Dabbler_Once_in_a_While: 9472.0
Dabbler_Two_Face: 9996.0
Dabbler_Accountant: 9881.0
Dabbler_Nervous: 8410.0
Dabbler_Collector: 8726.0
Coin_Flipper_Believer: 9506.0
Coin_Flipper_Once_in_a_While: 9808.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9980.0
Coin_Flipper_Nervous: 9479.0
Coin_Flipper_Collect

In [26]:
for col, val in results.loc["Bartek_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8289.0
Mat_Player_ver_4: 5015.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9997.0
Random_Never_Accuse: 9910.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5966.0
Saint_Once_in_a_While: 7828.0
Saint_Two_Face: 9971.0
Saint_Accountant: 9430.0
Saint_Nervous: 4961.0
Mat_Player_ver_1: 5011.0
Mat_Player_ver_2: 5034.0
Mat_Player_ver_3: 7532.0
Adam_M_Player: 9987.0
AdamK_ver_1: 6387.0
AdamK2_ver_2: 6778.0
Saint_Collector: 6050.0
Dabbler_Believer: 8672.0
Dabbler_Once_in_a_While: 9497.0
Dabbler_Two_Face: 9998.0
Dabbler_Accountant: 9873.0
Dabbler_Nervous: 8430.0
Dabbler_Collector: 8736.0
Coin_Flipper_Believer: 9470.0
Coin_Flipper_Once_in_a_While: 9807.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9976.0
Coin_Flipper_Nervous: 9491.0
Coin_Flipper_Collect

In [27]:
for col, val in results.loc["Bartek_ver_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 9297.0
Saint_Sceptic_25_Naive: 9257.0
Saint_Sceptic_50_Naive: 9246.0
Saint_Sceptic_75_Naive: 9248.0
Saint_Sceptic_Adaptable_Naive: 9740.0
Mat_Player_ver_4: 4711.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9309.0
MyRandom_All_Cards: 9951.0
MyRandom_Real_Cards: 7954.0
Random_Never_Accuse: 9709.0
Random_Always_Accuse: 8025.0
Mimic: 9930.0
Saint_Believer: 7807.0
Saint_Once_in_a_While: 8142.0
Saint_Two_Face: 9349.0
Saint_Accountant: 9642.0
Saint_Nervous: 6334.0
Mat_Player_ver_1: 5390.0
Mat_Player_ver_2: 5497.0
Mat_Player_ver_3: 5410.0
Adam_M_Player: 9387.0
AdamK_ver_1: 5384.0
AdamK2_ver_2: 5784.0
Saint_Collector: 7737.0
Dabbler_Believer: 9791.0
Dabbler_Once_in_a_While: 9471.0
Dabbler_Two_Face: 9507.0
Dabbler_Accountant: 9800.0
Dabbler_Nervous: 7891.0
Dabbler_Collector: 9745.0
Coin_Flipper_Believer: 9732.0
Coin_Flipper_Once_in_a_While: 9430.0
Coin_Flipper_Two_Face: 9299.0
Coin_Flipper_Accountant: 9703.0
Coin_Flipper_Nervous: 8027.0
Coin_Flipper_Collector: 967

In [28]:
for col, val in results.loc["Bartek_ver_4"].items():
    print(f'{col}: {val}')

RandomPlayer: 9255.0
Saint_Sceptic_25_Naive: 9268.0
Saint_Sceptic_50_Naive: 9241.0
Saint_Sceptic_75_Naive: 9318.0
Saint_Sceptic_Adaptable_Naive: 9715.0
Mat_Player_ver_4: 4589.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9182.0
MyRandom_All_Cards: 9939.0
MyRandom_Real_Cards: 7507.0
Random_Never_Accuse: 9719.0
Random_Always_Accuse: 8041.0
Mimic: 9923.0
Saint_Believer: 7798.0
Saint_Once_in_a_While: 8154.0
Saint_Two_Face: 9382.0
Saint_Accountant: 9630.0
Saint_Nervous: 6403.0
Mat_Player_ver_1: 5569.0
Mat_Player_ver_2: 5564.0
Mat_Player_ver_3: 5492.0
Adam_M_Player: 9347.0
AdamK_ver_1: 5322.0
AdamK2_ver_2: 5772.0
Saint_Collector: 7730.0
Dabbler_Believer: 9761.0
Dabbler_Once_in_a_While: 9486.0
Dabbler_Two_Face: 9496.0
Dabbler_Accountant: 9820.0
Dabbler_Nervous: 7956.0
Dabbler_Collector: 9725.0
Coin_Flipper_Believer: 9734.0
Coin_Flipper_Once_in_a_While: 9431.0
Coin_Flipper_Two_Face: 9236.0
Coin_Flipper_Accountant: 9713.0
Coin_Flipper_Nervous: 8092.0
Coin_Flipper_Collector: 967

In [29]:
for col, val in results.loc["Mat_Player_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8211.0
Mat_Player_ver_4: 5024.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9999.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9901.0
Random_Always_Accuse: 10000.0
Mimic: 9630.0
Saint_Believer: 5874.0
Saint_Once_in_a_While: 7621.0
Saint_Two_Face: 9966.0
Saint_Accountant: 9428.0
Saint_Nervous: 4905.0
Mat_Player_ver_1: nan
Mat_Player_ver_2: 4995.0
Mat_Player_ver_3: 7523.0
Adam_M_Player: 9988.0
AdamK_ver_1: 6354.0
AdamK2_ver_2: 6297.0
Saint_Collector: 5888.0
Dabbler_Believer: 8465.0
Dabbler_Once_in_a_While: 9329.0
Dabbler_Two_Face: 9995.0
Dabbler_Accountant: 9829.0
Dabbler_Nervous: 8194.0
Dabbler_Collector: 8533.0
Coin_Flipper_Believer: 9408.0
Coin_Flipper_Once_in_a_While: 9768.0
Coin_Flipper_Two_Face: 9998.0
Coin_Flipper_Accountant: 9969.0
Coin_Flipper_Nervous: 9387.0
Coin_Flipper_Collector: 9

In [30]:
for col, val in results.loc["Mat_Player_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 9998.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8164.0
Mat_Player_ver_4: 5011.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9999.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9906.0
Random_Always_Accuse: 10000.0
Mimic: 9614.0
Saint_Believer: 5898.0
Saint_Once_in_a_While: 7668.0
Saint_Two_Face: 9969.0
Saint_Accountant: 9403.0
Saint_Nervous: 5071.0
Mat_Player_ver_1: 5005.0
Mat_Player_ver_2: nan
Mat_Player_ver_3: 7426.0
Adam_M_Player: 9993.0
AdamK_ver_1: 6495.0
AdamK2_ver_2: 6311.0
Saint_Collector: 5967.0
Dabbler_Believer: 8430.0
Dabbler_Once_in_a_While: 9377.0
Dabbler_Two_Face: 9997.0
Dabbler_Accountant: 9831.0
Dabbler_Nervous: 8184.0
Dabbler_Collector: 8554.0
Coin_Flipper_Believer: 9422.0
Coin_Flipper_Once_in_a_While: 9779.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9967.0
Coin_Flipper_Nervous: 9410.0
Coin_Flipper_Collector: 9

In [31]:
for col, val in results.loc["Mat_Player_ver_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 10000.0
Saint_Sceptic_25_Naive: 9086.0
Saint_Sceptic_50_Naive: 9245.0
Saint_Sceptic_75_Naive: 9194.0
Saint_Sceptic_Adaptable_Naive: 5818.0
Mat_Player_ver_4: 2545.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9998.0
Random_Never_Accuse: 9908.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5020.0
Saint_Once_in_a_While: 5587.0
Saint_Two_Face: 9222.0
Saint_Accountant: 6471.0
Saint_Nervous: 2478.0
Mat_Player_ver_1: 2477.0
Mat_Player_ver_2: 2574.0
Mat_Player_ver_3: nan
Adam_M_Player: 9159.0
AdamK_ver_1: 4663.0
AdamK2_ver_2: 4648.0
Saint_Collector: 5239.0
Dabbler_Believer: 8020.0
Dabbler_Once_in_a_While: 9152.0
Dabbler_Two_Face: 9995.0
Dabbler_Accountant: 9730.0
Dabbler_Nervous: 7459.0
Dabbler_Collector: 8126.0
Coin_Flipper_Believer: 9350.0
Coin_Flipper_Once_in_a_While: 9752.0
Coin_Flipper_Two_Face: 9997.0
Coin_Flipper_Accountant: 9958.0
Coin_Flipper_Nervous: 9316.0
Coin_Flipper_Collector: 9

In [32]:
for col, val in results.loc["Mat_Player_ver_4"].items():
    print(f'{col}: {val}')

RandomPlayer: 9998.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8310.0
Mat_Player_ver_4: nan
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9920.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 6021.0
Saint_Once_in_a_While: 7780.0
Saint_Two_Face: 9969.0
Saint_Accountant: 9434.0
Saint_Nervous: 5052.0
Mat_Player_ver_1: 4976.0
Mat_Player_ver_2: 4989.0
Mat_Player_ver_3: 7455.0
Adam_M_Player: 9988.0
AdamK_ver_1: 6458.0
AdamK2_ver_2: 6821.0
Saint_Collector: 6025.0
Dabbler_Believer: 8628.0
Dabbler_Once_in_a_While: 9483.0
Dabbler_Two_Face: 10000.0
Dabbler_Accountant: 9875.0
Dabbler_Nervous: 8405.0
Dabbler_Collector: 8708.0
Coin_Flipper_Believer: 9517.0
Coin_Flipper_Once_in_a_While: 9810.0
Coin_Flipper_Two_Face: 10000.0
Coin_Flipper_Accountant: 9976.0
Coin_Flipper_Nervous: 9482.0
Coin_Flipper_Collecto

In [33]:
for col, val in results.loc["Samuel_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9791.0
Saint_Sceptic_25_Naive: 9591.0
Saint_Sceptic_50_Naive: 9592.0
Saint_Sceptic_75_Naive: 9623.0
Saint_Sceptic_Adaptable_Naive: 8006.0
Mat_Player_ver_4: 5937.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9977.0
MyRandom_All_Cards: 9994.0
MyRandom_Real_Cards: 9742.0
Random_Never_Accuse: 8542.0
Random_Always_Accuse: 9983.0
Mimic: 10000.0
Saint_Believer: 6894.0
Saint_Once_in_a_While: 7151.0
Saint_Two_Face: 8876.0
Saint_Accountant: 8705.0
Saint_Nervous: 6027.0
Mat_Player_ver_1: 5984.0
Mat_Player_ver_2: 5936.0
Mat_Player_ver_3: 6260.0
Adam_M_Player: 9640.0
AdamK_ver_1: 5815.0
AdamK2_ver_2: 6652.0
Saint_Collector: 6844.0
Dabbler_Believer: 7889.0
Dabbler_Once_in_a_While: 8158.0
Dabbler_Two_Face: 9652.0
Dabbler_Accountant: 9210.0
Dabbler_Nervous: 6827.0
Dabbler_Collector: 7912.0
Coin_Flipper_Believer: 8499.0
Coin_Flipper_Once_in_a_While: 8910.0
Coin_Flipper_Two_Face: 9794.0
Coin_Flipper_Accountant: 9806.0
Coin_Flipper_Nervous: 8003.0
Coin_Flipper_Collector: 86

In [34]:
for col, val in results.loc["Pukacki_Tabaszewski"].items():
    print(f'{col}: {val}')

RandomPlayer: 10000.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8325.0
Mat_Player_ver_4: 5033.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9998.0
Random_Never_Accuse: 9927.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5970.0
Saint_Once_in_a_While: 7729.0
Saint_Two_Face: 9957.0
Saint_Accountant: 9458.0
Saint_Nervous: 4943.0
Mat_Player_ver_1: 4920.0
Mat_Player_ver_2: 4993.0
Mat_Player_ver_3: 7462.0
Adam_M_Player: 9988.0
AdamK_ver_1: 6550.0
AdamK2_ver_2: 6770.0
Saint_Collector: 6017.0
Dabbler_Believer: 8626.0
Dabbler_Once_in_a_While: 9489.0
Dabbler_Two_Face: 9997.0
Dabbler_Accountant: 9886.0
Dabbler_Nervous: 8465.0
Dabbler_Collector: 8741.0
Coin_Flipper_Believer: 9511.0
Coin_Flipper_Once_in_a_While: 9837.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9976.0
Coin_Flipper_Nervous: 9470.0
Coin_Flipper_Collec

### Results - Only Student's Bots

In [35]:
agent_strategies = [
            ["Mat_Player_ver_4", "ENSAMBLE", ensamble_player],
           ["Mat_Player_ver_1", "SAINT", Saint_Nervous_ver_2],
           ["Mat_Player_ver_2", "SAINT", Saint_Nervous_ver_3],
           ["Mat_Player_ver_3", "SAINT", Saint_Nervous_Memory],
           ["Adam_M_Player", "CUSTOM", Adam_player],
           ["AdamK_ver_1", "CUSTOM", AdamK],
           ["AdamK2_ver_2", "CUSTOM", AdamK2],
           ["Bartek_ver_1", "B_PLAYER", Saint_Nervous_ver_B],
           ["Bartek_ver_2", "B_PLAYER", Saint_Nervous_ver_B2],
           ["Bartek_ver_3", "B_PLAYER", Saint_Nervous_ver_B3],
           ["Bartek_ver_4", "B_PLAYER", Saint_Nervous_ver_B4],
           ["Janek_Michal_ver_1", "MICHAL_JANEK", KaminskiIndrzejczak_ver_1],
           ["Janek_Michal_ver_2", "MICHAL_JANEK", KaminskiIndrzejczak_ver_2],
           ["Vadym_Player", "VADYM", Vadym_Player],
           ["Krzysztof_Player", "KRZYSZTOF", Krzysztof_Player],
           ["Samuel_Player", "SAMUEL", Amalgamat],
           ["Pukacki_Tabaszewski", "PUKACKI_TABASZEWSKI", Pukacki_Tabaszewski]
           ]

results = compare_all(agent_strategies)
results

,Mat_Player_ver_4,Mat_Player_ver_1,Mat_Player_ver_2,Mat_Player_ver_3,Adam_M_Player,AdamK_ver_1,AdamK2_ver_2,Bartek_ver_1,Bartek_ver_2,Bartek_ver_3,Bartek_ver_4,Janek_Michal_ver_1,Janek_Michal_ver_2,Vadym_Player,Krzysztof_Player,Samuel_Player,Pukacki_Tabaszewski,|| Total Wins ||,|| Dominated Players ||
Samuel_Player,5913.0,5902.0,5868.0,6244.0,9588.0,5713.0,6672.0,5923.0,5961.0,5751.0,5755.0,9742.0,4712.0,6361.0,9595.0,NaN,5925.0,105625.0,15
Bartek_ver_1,5052.0,5011.0,4973.0,7465.0,9995.0,6548.0,6791.0,NaN,5007.0,5359.0,5360.0,10000.0,3992.0,9187.0,10000.0,4077.0,4948.0,103765.0,12
Janek_Michal_ver_2,6032.0,5966.0,6007.0,6126.0,7966.0,5924.0,6653.0,6008.0,6006.0,6012.0,5964.0,9456.0,NaN,5179.0,9125.0,5288.0,6048.0,103760.0,16
Pukacki_Tabaszewski,5041.0,5030.0,4905.0,7480.0,9988.0,6532.0,6676.0,5052.0,5029.0,5275.0,5411.0,10000.0,3952.0,9227.0,10000.0,4075.0,NaN,103673.0,13
Mat_Player_ver_4,NaN,4974.0,4996.0,7522.0,9993.0,6462.0,6866.0,4948.0,5044.0,5344.0,5304.0,10000.0,3968.0,9196.0,10000.0,4087.0,4959.0,103663.0,10
Bartek_ver_2,4956.0,4974.0,5013.0,7474.0,9989.0,6507.0,6690.0,4993.0,NaN,5306.0,5346.0,10000.0,3994.0,9208.0,10000.0,4039.0,4971.0,103460.0,10
Mat_Player_ver_1,5026.0,NaN,5109.0,7458.0,9986.0,6476.0,6344.0,4989.0,5026.0,4496.0,4518.0,10000.0,4034.0,9151.0,10000.0,4098.0,4970.0,101681.0,10
Mat_Player_ver_2,5004.0,4891.0,NaN,7511.0,9990.0,6448.0,6204.0,5027.0,4987.0,4498.0,4585.0,10000.0,3993.0,9179.0,10000.0,4132.0,5095.0,101544.0,10
Bartek_ver_3,4656.0,5504.0,5502.0,5403.0,9393.0,5347.0,5868.0,4641.0,4694.0,NaN,5014.0,9643.0,3988.0,3888.0,9320.0,4249.0,4725.0,91835.0,9
Bartek_ver_4,4696.0,5482.0,5415.0,5377.0,9367.0,5290.0,5802.0,4640.0,4654.0,4986.0,NaN,9545.0,4036.0,3881.0,9250.0,4245.0,4589.0,91255.0,8
